In [ ]:
!pip3 install optuna
!pip3 install transformers

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import logging
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import optuna

In [2]:
import transformers
from transformers import BertTokenizer
token = BertTokenizer.from_pretrained('bert-base-chinese')

In [3]:
textpath = '/openbayes/input/input0/text'
outpath = '/openbayes/input/input0/text_encoded'
def get_list(datapath, outpath):
    # 获取待处理的list
    try:
        datalist = os.listdir(datapath)
        donelist = os.listdir(outpath)
        tohandle_list = [x for x in datalist if x not in donelist]
        tohandle_list = [x for x in tohandle_list if x[-4:] == '.csv']
        return tohandle_list, donelist
    except:
        return False

In [4]:
tohandle_list, donelist = get_list(textpath, outpath)

In [5]:
print(len(tohandle_list), len(donelist))

80

In [7]:
import torch
from transformers import BertModel

class MLP(nn.Module):
    def __init__(self, input_size, hidden_layer_sizes, output_size):
        super(MLP, self).__init__()
        # 初始化权重
        self.initializer = nn.init.kaiming_normal_
        layers = [nn.Linear(input_size, hidden_layer_sizes[0])]  # 第一个隐藏层
        # 添加ReLU激活函数
        layers.append(nn.ReLU())
        # 添加更多的隐藏层
        for i in range(1, len(hidden_layer_sizes)):
            layers.append(nn.Linear(hidden_layer_sizes[i-1], hidden_layer_sizes[i]))
            layers.append(nn.ReLU())
        # 最后一个隐藏层到输出层
        layers.append(nn.Linear(hidden_layer_sizes[-1], output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                self.initializer(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

class CommentBertEncode(object):
    def __init__(self, textpath, outpath, stockname, num=500):
        self.outpath = outpath
        self.textpath = textpath
        self.stockname = stockname
        self.logger = None
        self.token = BertTokenizer.from_pretrained('bert-base-chinese')
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = 'cuda'
        self.num = num

    def logger_config(self, log_path, logging_name):
        '''
        配置log
        :param log_path: 输出log路径
        :param logging_name: 记录中name，可随意
        :return:
        '''
        '''
        logger是日志对象，handler是流处理器，console是控制台输出（没有console也可以，将不会在控制台输出，会在日志文件中输出）
        '''
        # 获取logger对象,取名
        logger = logging.getLogger(logging_name)
        # 输出DEBUG及以上级别的信息，针对所有输出的第一层过滤
        logger.setLevel(level=logging.DEBUG)
        if not logger.handlers:
            # 获取文件日志句柄并设置日志级别，第二层过滤
            handler = logging.FileHandler(log_path, encoding='UTF-8')
            handler.setLevel(logging.INFO)
            # 生成并设置文件日志格式
            formatter = logging.Formatter('%(message)s - %(levelname)s - %(asctime)s - %(name)s')
            handler.setFormatter(formatter)
            # 为logger对象添加句柄
            logger.addHandler(handler)
            # logger.addHandler(console)
        return logger

    
        
    def get_text(self):
        # 获取文本
        text_df = pd.read_csv(os.path.join(self.textpath,self.stockname))
        text_l = text_df['text'].tolist()
        
        return text_l
    
    def collate_fn(self, data):
        #编码
        datas = self.token.batch_encode_plus(batch_text_or_text_pairs=data,
                                       truncation=True,
                                       padding='max_length',
                                       max_length=100,
                                       return_tensors='pt',
                                       return_length=True)

        #input_ids:编码之后的数字
        #attention_mask:是补零的位置是0,其他位置是1
        input_ids = datas['input_ids']
        attention_mask = datas['attention_mask']
        token_type_ids = datas['token_type_ids']

        return input_ids, attention_mask, token_type_ids
    
    def encode_text(self, input_ids, attention_mask, token_type_ids):
        num = self.num
        device = self.device
        input_size = 768  # 输入特征维度
        hidden_layer_sizes = [188, 158, 79]  # 假设有3个隐藏层
        output_size = 2  # 假设是2分类问题
        learning_rate = 0.00020148404515308536
        optimizer ="Adam"
        # 创建一个新的模型实例
        new_model = MLP(input_size, hidden_layer_sizes, output_size)
        # 加载状态字典
        new_model.load_state_dict(torch.load('best_model_state_dict.pth'))
        # 确保模型在相同的设备上
        new_model.to(device)
        new_model.eval()

        for i in range(0, len(input_ids), num):
            # 加载预训练模型
            pretrained = BertModel.from_pretrained('bert-base-chinese')
            # 不训练预训练模型,不需要计算梯度
            for param in pretrained.parameters():
                param.requires_grad_(False)
            pretrained.to(device)
            
            if len(input_ids) - i > num:
                input_ids1 = input_ids[i:i+num].to(device)
                attention_mask1 = attention_mask[i:i+num].to(device)
                token_type_ids1 = token_type_ids[i:i+num].to(device)
            else:
                input_ids1 = input_ids[i:].to(device)
                attention_mask1 = attention_mask[i:].to(device)
                token_type_ids1 = token_type_ids[i:].to(device)
            out = pretrained(input_ids1, attention_mask1, token_type_ids1)
            out = out.last_hidden_state[:, 0].cpu().detach().numpy().tolist()
            
            X_tensor = torch.tensor(out, dtype=torch.float32)
            predictions = new_model(X_tensor.to(device))
            # predictions = new_model(X_tensor.to(device)).argmax(dim=1).cpu().tolist()
            # 使用 softmax 获取概率分布
            probabilities = F.softmax(predictions, dim=1).cpu().tolist()
            if i == 0:
                prob = probabilities
            else:     
                prob.extend(probabilities)
            # input_ids1.cpu() 
            # attention_mask1.cpu() 
            # token_type_ids1.cpu()
            # pretrained.cpu()
            del input_ids1, attention_mask1, token_type_ids1, out, pretrained
            torch.cuda.empty_cache()
            allocated_memory = torch.cuda.memory_allocated()
            cached_memory = torch.cuda.memory_reserved()
            # print(f"已分配的GPU内存：{allocated_memory}, 已缓存的GPU内存：{cached_memory}")
        print(np.shape(prob))
        return prob
        
    def save_data(self, out):
        try:
            np.savetxt(os.path.join(self.outpath, self.stockname), out, delimiter=",")
            return True   
        except:
            return False

    def RunStart(self):
        self.logger = self.logger_config(log_path=os.path.join(self.outpath, 'log.txt'), logging_name='hpz')
        text = self.get_text()
        if text == []:
            self.logger.error(f"{self.stockname}读取失败!")
            return
        self.logger.info(f"{self.stockname}开始处理!")
        try:
            input_ids, attention_mask, token_type_ids = self.collate_fn(text) 
        except:
            self.logger.error(f"{self.stockname}初编码失败!")
            return
        self.logger.info(f"{self.stockname}初编码完成!")
        try:
            out = self.encode_text(input_ids, attention_mask, token_type_ids)
        except:
            self.logger.error(f"{self.stockname}编码失败!")
            return
        self.logger.info(f"{self.stockname}编码完成!")
        if self.save_data(out):
            self.logger.info(f"{self.stockname}处理完成!")
        else:
            self.logger.info(f"{self.stockname}处理失败!")
        del input_ids, attention_mask, token_type_ids, out
        logging.shutdown()
        return

In [8]:
for stockname in tqdm(tohandle_list):
    encode = CommentBertEncode(textpath, outpath, stockname, num=2800)
    encode.RunStart()
    del encode

  0%|          | 0/80 [00:00<?, ?it/s]

(206331, 2)


  1%|▏         | 1/80 [04:02<5:19:35, 242.72s/it]

(385843, 2)


  2%|▎         | 2/80 [11:31<7:52:55, 363.78s/it]

...

 96%|█████████▋| 77/80 [5:07:11<08:11, 163.96s/it]

(138572, 2)


 99%|█████████▉| 79/80 [5:11:05<02:16, 136.42s/it]

(62842, 2)
(71140, 2)


100%|██████████| 80/80 [5:12:30<00:00, 234.38s/it]
